# Sprint セグメンテーション１

## このSprintについて
### Sprintの目的
* コンピュータビジョンの代表的タスクであるセグメンテーションを学ぶ

### どのように学ぶか
公開されている実装を用いてKaggleコンペティションのデータでセグメンテーションを行います。

### セグメンテーション
KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーションを行います。

TGS Salt Identification Challenge | Kaggle  
https://www.kaggle.com/c/tgs-salt-identification-challenge


セグメンテーション手法としてU-Net[1]を使います。

[1]O.Ronneberger, P.Fischer, T.Brox. U-Net: Convolutional Networks for Biomedical Image Segmentation. Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234–241, 2015

https://arxiv.org/pdf/1505.04597.pdf

### 【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。

zhixuhao/unet: unet for image segmentation  
https://github.com/zhixuhao/unet

《GPU環境での学習》

大規模なデータセット、大きなモデルになるため、GPUを使用する必要があります。

《新たなデータセットの適用》

公開されている実装で用意されたものとは異なるデータセットを入力するための準備が必要です。

In [2]:
from model import *
from data import *

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [3]:
data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')

myGene = trainGenerator(20,
                        'data/membrane/train',
                        'image',
                        'label',
                        data_gen_args,
                        save_to_dir = None)

model = unet()
model_checkpoint = ModelCheckpoint('unet_salt.hdf5',
                                   monitor='loss',
                                   verbose=1,
                                   save_best_only=True)

model.fit_generator(myGene,
                    steps_per_epoch=10,
                    epochs=5,
                    callbacks=[model_checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Found 490 images belonging to 1 classes.
Found 490 images belonging to 1 classes.
Epoch 1/5
10/10 [==============================] - ETA: 0s - loss: 0.6877 - accuracy: 0.7333  
Epoch 00001: loss improved from inf to 0.68771, saving model to unet_salt.hdf5
10/10 [==============================] - 1741s 174s/step - loss: 0.6877 - accuracy: 0.7333
Epoch 2/5
10/10 [==============================] - ETA: 0s - loss: 0.5566 - accuracy: 0.7561  
Epoch 00002: loss improved from 0.68771 to 0.55663, saving model to unet_salt.hdf5
10/10 [==============================] - 1732s 173s/step - loss: 0.5566 - accuracy: 0.7561
Epoch 3/5
10/10 [==============================] - ETA: 0s - loss: 0.5820 - accuracy: 0.7604  
Epoch 00003: loss did not improve from 0.55663
10/10 [==============================] - 1145s 114s/step - loss: 0.5820 - accuracy: 0.7604
Epoch 4/5
10/10 [==============================] - ETA: 0s - loss: 0.5739 -

### Train with npy file

In [4]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [8]:
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_salt.hdf5")
results = model.predict_generator(testGene,10,verbose=1)
saveResult("data/membrane/test",results)

10/10 [==============================] - 16s 2s/step


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/kazuki/diveintocode-ml/スプリント/unet-master/data.py:126: UserWarning: data/membrane/test/1_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/kazuki/diveintocode-ml/スプリント/unet-master/data.py:126: UserWarning: data/membrane/test/2_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/kazuki/diveintocode-ml/スプリント/unet-master/data.py:126: UserWarning: data/membrane/test/3_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to

### 【問題2】コードリーディング
論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。

* U-Netの構成
1. Affine layerがない
2. downsampling(3x3conv 2回 --> ReLU --> 2x2maxpooling)を行う。feature map が2倍ずつ増える
3. upsampling